# Mass and isotopes

MolecularGraph.jl version: 0.10.0

This tutorial includes following fundamental operations related to molecular mass and isotopes.

- Standard molecular weight
- Relative molecular mass (Calculated exact mass)
- Monoisotopic mass
- Isotopic composition
- Simulate mass spectrum

## Loading packages

This tutorial uses `Plots.jl` to display simulated mass spectrum.

In [1]:
using Pkg
Pkg.activate("..")
using MolecularGraph
using Plots

# To display a pair of mol images
function displayimgpair(img1, img2)
    """<div>
        <div style="float:left">$(img1)</div>
        <div style="float:left">$(img2)</div>
    </div>"""
end

nothing

  Activating environment at `~/Repository/MolecularGraph.jl_notebook/Project.toml`
Path `../../MolecularGraph.jl` exists and looks like the correct package. Using existing path.
   Resolving package versions...
  No Changes to `~/Repository/MolecularGraph.jl_notebook/Project.toml`
  No Changes to `~/Repository/MolecularGraph.jl_notebook/Manifest.toml`


## Molecular weight (Relative molecular mass)

Molecular weight (Relative molecular mass) can be calculated by `standardweight` method.

In [2]:
standardweight(smilestomol("CCO"))

(46.069, 0.004529999999995926)

- `standardweight` returns the tuple of standard weight and its uncertainty, respectively. The result shown above means the molecular weight of C2H5O is 46.069±0.005.
- If you dont need uncertainty, give Float64 type to the method. Then the value will be rounded at 2 digits after the decimal point.

In [3]:
standardweight(Float64, smilestomol("CCO"))

46.07

## Calculated exact mass and monoisotopic mass

Both `exactmass` and `monoisotopicmass` returns exact mass of the given molecule. The difference is that `exactmass` considers atomic mass specified in `Atom.mass` fields of atoms whereas `monoisotopicmass` always returns exact mass of the molecule consists of the most abundant isotopes in terrestrial sources.

In [4]:
etohd6 = smilestomol("[2H]C([2H])([2H])C([2H])([2H])O[2H]")

println("Molecular weight: ", standardweight(Float64, etohd6))
println("Monoisotopic: ", monoisotopicmass(Float64, etohd6))
println("Exact: ", exactmass(Float64, etohd6))

Molecular weight: 52.11
Monoisotopic: 46.041865
Exact: 52.079525


## Isotopic composition

`isotopiccomposition(atomsymbol, num)` returns total masses of the given number of atoms and their isotopic compositions.

Isotopes that have lower abundance than the `threshold` parameter will be filtered out (default 0.001 = 0.1%)

In [5]:
isotopiccomposition(:C, 100; threshold=0.01)

5-element Vector{Tuple{Float64, Float64}}:
 (1204.01341934028, 0.01829972319797687)
 (1203.01006450521, 0.06977096630391633)
 (1202.00670967014, 0.19747098047720194)
 (1201.00335483507, 0.36875136628763155)
 (1200.0, 0.3410365354703909)

In [6]:
isotopiccomposition(:C, 1000; threshold=0.01)

15-element Vector{Tuple{Float64, Float64}}:
 (12018.06038703126, 0.011690519214538612)
 (12017.05703219619, 0.019792334300846457)
 (12016.05367736112, 0.0316151371989973)
 (12015.05032252605, 0.04748134937359465)
 (12014.04696769098, 0.06678541451097929)
 (12013.04361285591, 0.08758641222522136)
 (12012.04025802084, 0.1065534144511583)
 (12011.03690318577, 0.11953540283428146)
 (12010.0335483507, 0.12279996975578453)
 (12009.03019351563, 0.11456944727429401)
 (12008.02683868056, 0.09610452890364467)
 (12007.02348384549, 0.0715861101895721)
 (12006.02012901042, 0.04661058275253684)
 (12005.01677417535, 0.02598703123363034)
 (12004.01341934028, 0.012061765298138035)

`isotopiccomposition(mol)` returns isotopic composition of the molecule.

In [7]:
isotopiccomposition(smilestomol("CCO"))

3-element Vector{Tuple{Float64, Float64}}:
 (46.041864812949996, 0.9756627354527866)
 (47.045219648019994, 0.02109374866472255)
 (48.04610980624, 0.0020049807108054704)

## Simulate mass spectrum

According to the isotopic composition, simulated mass spectrum can be plotted by using `simulatemassspec` method and available plot libraries.
Follwing example uses Plots.jl to draw the simulated spectrum.

In [8]:
mol = smilestomol("c1cc(ccc1[C@H]([C@@H](CO)NC(=O)C(Cl)Cl)O)[N+](=O)[O-]")
structure = drawsvg(mol, 200, 200)
data = simulatemassspec(mol)
p = plot(data[:, 1], data[:, 2], size=(320, 240), leg=false, xlabel = "Mass", ylabel = "Intensity")
buf = IOBuffer()
show(buf, MIME("image/svg+xml"), p)
spectrum = String(take!(buf))
close(buf)
display("text/html", displayimgpair(structure, spectrum))

N + 
 HO 
 H 
 Cl 
 HO 
 O – 
 NH 
 O 
 H 
 Cl 
 O 
 
 
 <?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<path clip-path="url(#clip190)" d="M365.68 812.398 Q369.036 813.115 370.911 815.384 Q372.81 817.652 372.81 820.986 Q372.81 826.101 369.291 828.902 Q365.773 831.703 359.291 831.703 Q357.115 831.703 354.8 831.263 Q352.509 830.847 350.055 829.99 L350.055 825.476 Q352 826.611 354.314 827.189 Q356.629 827.768 359.152 827.768 Q363.55 827.768 365.842 826.032 Q368.157 824.296 368.157 820.986 Q368.157 817.93 366.004 816.217 Q363.874 814.481 360.055 814.481 L356.027 814.481 L356.027 810.638 L360.24 810.638 Q363.689 810.638 365.518 809.273 Q367.347 807.884 367.347 805.291 Q367.347 802.629 365.449 801.217 Q363.574 799.782 360.055 799.782 Q358.134 799.782 355.935 800.199 Q353.736 800.615 351.097 801.495 L351.097 797.328 Q353.759 796.588 356.074 796.217 Q358.412 795.847 360.472 795.847 Q365.796 795.847 368.898 798.277 Q371.999 800.685 371.999 804.805 Q371.999 807.675 370.356 809.666 Q368.712 811.634 365.68 812.398 Z" fill="#000000" fill-rule="evenodd" fill-opacity="1" /><path clip-path="url(#clip190)" d="M385.703 827.097 L402.022 827.097 L402.022 831.032 L380.078 831.032 L380.078 827.097 Q382.74 824.342 387.323 819.712 Q391.93 815.06 393.11 813.717 Q395.356 811.194 396.235 809.458 Q397.138 807.699 397.138 806.009 Q397.138 803.254 395.194 801.518 Q393.272 799.782 390.171 799.782 Q387.972 799.782 385.518 800.546 Q383.087 801.31 380.31 802.861 L380.31 798.139 Q383.134 797.004 385.587 796.426 Q388.041 795.847 390.078 795.847 Q395.448 795.847 398.643 798.532 Q401.837 801.217 401.837 805.708 Q401.837 807.838 401.027 809.759 Q400.24 811.657 398.134 814.25 Q397.555 814.921 394.453 818.138 Q391.351 821.333 385.703 827.097 Z" fill="#000000" fill-rule="evenodd" fill-opacity="1" /><path clip-path="url(#clip190)" d="M415.865 827.097 L432.184 827.097 L432.184 831.032 L410.24 831.032 L410.24 827.097 Q412.902 824.342 417.485 819.712 Q422.092 815.06 423.272 813.717 Q425.518 811.194 426.397 809.458 Q427.3 807.699 427.3 806.009 Q427.3 803.254 425.356 801.518 Q423.434 799.782 420.332 799.782 Q418.133 799.782 415.68 800.546 Q413.249 801.31 410.471 802.861 L410.471 798.139 Q413.295 797.004 415.749 796.426 Q418.203 795.847 420.24 795.847 Q425.61 795.847 428.805 798.532 Q431.999 801.217 431.999 805.708 Q431.999 807.838 431.189 809.759 Q430.402 811.657 428.295 814.25 Q427.717 814.921 424.615 818.138 Q421.513 821.333 415.865 827.097 Z" fill="#000000" fill-rule="evenodd" fill-opacity="1" /><path clip-path="url(#clip190)" d="M635.697 812.398 Q639.053 813.115 640.928 815.384 Q642.826 817.652 642.826 820.986 Q642.826 826.101 639.308 828.902 Q635.789 831.703 629.308 831.703 Q627.132 831.703 624.817 831.263 Q622.525 830.847 620.072 829.99 L620.072 825.476 Q622.016 826.611 624.331 827.189 Q626.646 827.768 629.169 827.768 Q633.567 827.768 635.859 826.032 Q638.174 824.296 638.174 820.986 Q638.174 817.93 636.021 816.217 Q633.891 814.481 630.072 814.481 L626.044 814.481 L626.044 810.638 L630.257 810.638 Q633.706 810.638 635.535 809.273 Q637.363 807.884 637.363 805.291 Q637.363 802.629 635.465 801.217 Q633.59 799.782 630.072 799.782 Q628.15 799.782 625.951 800.199 Q623.752 800.615 621.113 801.495 L621.113 797.328 Q623.775 796.588 626.09 796.217 Q628.428 795.847 630.488 795.847 Q635.812 795.847 638.914 798.277 Q642.016 800.685 642.016 804.805 Q642.016 807.675 640.373 809.666 Q638.729 811.634 635.697 812.398 Z" fill="#000000" fill-rule="evenodd" fill-opacity="1" /><path clip-path="url(#clip190)" d="M655.72 827.097 L672.039 827.097 L672.039 831.032 L650.095 831.032 L650.095 827.097 Q652.757 824.342 657.34 819.712 Q661.947 815.06 663.127 813.717 Q665.372 811.194 666.252 809.458 Q667.155 807.699 667.155 806.009 Q667.155 803.254 665.21 801.518 Q663.289 799.782 660.187 799.782 Q657.988 799.782 655.535 800.546 Q653.104 801.31 650.326 802.861 L650.326 798.139 Q653.15 797.004 655.604 796.426 Q658.058 795.847 660.095 795.847 Q665.465 795.847